In [ ]:
#import some libraries
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')
# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")
print("Path to dataset files:", path)

In [ ]:
# config visualization
plt.style.use('ggplot')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
#Check the download folder's content.
import os
#list all files 
print(os.listdir(path))

In [ ]:
#Use pandas for read the file 
import pandas as pd
#Build the full path to the CSV file
csv_path = os.path.join(path,"WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = pd.read_csv(csv_path)
df.head()#check the data 

## 2. ANÁLISIS EXPLORATORIO DE DATOS (EDA)

In [ ]:

# basic information 
print(df.info())

In [ ]:
# Descriptive statistic
print(df.describe())


In [ ]:
# Check null values
print("Null value per columns:")
print(df.isnull().sum())

In [ ]:
#Checking unique categorical values 
print('Unique categorical values on columns')
categorical_cols = df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    print(f"{col}: {df[col].unique()}")
    print("-" * 30)

In [ ]:
#convert TotalChange to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'] ,errors='coerce')

#check the null values after 
print(f"null values in Total Changes is:{df['TotalCharges'].isnull().sum()}")


null values in Total Changes is:11


In [17]:
#Eliminate the nan values
df = df.dropna()